# Access S2S via Python Library

In [1]:
%%capture
%pip install "git+https://github.com/worldbank/DECAT_Space2Stats.git#subdirectory=space2stats_api/src"

In [2]:
from typing import Dict

import numpy as np
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from lonboard import Map, ScatterplotLayer
from geojson_pydantic import Feature, Polygon

from space2stats import StatsTable

__Expected environment variables to be set__ from dotenv file


```.env
PGHOST=
PGPORT=
PGDATABASE=
PGUSER=
PGPASSWORD=
PGTABLENAME=space2stats
```

In [3]:
load_dotenv("../../db.env")

True

In [4]:
with StatsTable.connect() as stats_table:
    fields = stats_table.fields()

fields

['sum_pop_2020',
 'ogc_fid',
 'sum_pop_f_0_2020',
 'sum_pop_f_10_2020',
 'sum_pop_f_15_2020',
 'sum_pop_f_1_2020',
 'sum_pop_f_20_2020',
 'sum_pop_f_25_2020',
 'sum_pop_f_30_2020',
 'sum_pop_f_35_2020',
 'sum_pop_f_40_2020',
 'sum_pop_f_45_2020',
 'sum_pop_f_50_2020',
 'sum_pop_f_55_2020',
 'sum_pop_f_5_2020',
 'sum_pop_f_60_2020',
 'sum_pop_f_65_2020',
 'sum_pop_f_70_2020',
 'sum_pop_f_75_2020',
 'sum_pop_f_80_2020',
 'sum_pop_m_0_2020',
 'sum_pop_m_10_2020',
 'sum_pop_m_15_2020',
 'sum_pop_m_1_2020',
 'sum_pop_m_20_2020',
 'sum_pop_m_25_2020',
 'sum_pop_m_30_2020',
 'sum_pop_m_35_2020',
 'sum_pop_m_40_2020',
 'sum_pop_m_45_2020',
 'sum_pop_m_50_2020',
 'sum_pop_m_55_2020',
 'sum_pop_m_5_2020',
 'sum_pop_m_60_2020',
 'sum_pop_m_65_2020',
 'sum_pop_m_70_2020',
 'sum_pop_m_75_2020',
 'sum_pop_m_80_2020',
 'sum_pop_m_2020',
 'sum_pop_f_2020']

In [5]:
AOIModel = Feature[Polygon, Dict]

# ~kenya
aoi = {
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [33.78593974945852, 5.115816884114494],
                [33.78593974945852, -4.725410543134203],
                [41.94362577283266, -4.725410543134203],
                [41.94362577283266, 5.115816884114494],
                [33.78593974945852, 5.115816884114494]
            ]
        ]
    },
    "properties": {
        "name": "Updated AOI"
    }
}
          
  

feat = AOIModel(**aoi)

In [6]:
with StatsTable.connect() as stats_table:
    data = stats_table.summaries(aoi=feat, spatial_join_method="centroid", fields=fields, geometry="point")
    df = pd.DataFrame(data)

df.head()

AttributeError: module 'h3' has no attribute 'polyfill'

In [ ]:
df['geometry'] = df['geometry'].apply(lambda geom: shape(geom))
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf

In [ ]:
# Define custom breaks and corresponding RGBA colors
breaks = [0, 1, 1000, 10000, 50000, 100000, 200000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 255],  # Light gray for 0
    [255, 255, 0, 255],    # Yellow for 1-1000
    [255, 165, 0, 255],    # Orange for 1000-10000
    [255, 0, 0, 255],      # Red for 10000-50000
    [128, 0, 128, 255],    # Purple for 50000-100000
    [0, 0, 255, 255],      # Blue for 100000-200000
    [0, 0, 139, 255],      # Dark blue for 200000+
])

# Function to assign colors based on custom bins
def assign_color(value, breaks, colors):
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i]
    return colors[-1]  # In case value exceeds all breaks

# Map sum_pop_2020 values to colors using the custom function
gdf['color'] = gdf["sum_pop_2020"].apply(lambda x: assign_color(x, breaks, colors))
colors = np.uint8(gdf['color'].tolist())

# Create the scatterplot layer with the assigned colors
layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=colors)

m = Map(layer)
m